<a href="https://colab.research.google.com/github/tazar09/heroes_3/blob/main/h3_heroes%5B28.oct2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [66]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.options.display.max_rows = 300

# Importing data

In [2]:
df1 = pd.read_html('https://heroes.thelazy.net/index.php/List_of_heroes_(HotA)')
df1[0].head()

,Name,Class,Unnamed: 2,Specialty,Unnamed: 4,Skill 1,Unnamed: 6,Skill 2,Unnamed: 8,Spell
0,Anabel,Captain,NaN,Pirates,NaN,Basic Offense,NaN,Basic Archery,NaN,NaN
1,Cassiopeia,Captain,NaN,Nymphs,NaN,Basic Offense,NaN,Basic Tactics,NaN,NaN
2,Corkes,Captain,NaN,Offense,NaN,Basic Offense,NaN,Basic Pathfinding,NaN,NaN
3,Derek,Captain,NaN,Crew Mates,NaN,Basic Offense,NaN,Basic Leadership,NaN,NaN
4,Elmore,Captain,NaN,Navigation,NaN,Advanced Navigation,NaN,NaN,NaN,NaN


# Creating dataframe

## Data preprocessing

In [112]:
df = df1[0].copy()
df = df.drop(axis = 1, labels = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8'])
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell']

## Dropping and renaming

In [113]:
skill1 = df['skill_1'].str.split(expand = True, n = 1)
skill2 = df['skill_2'].str.split(expand = True, n = 1)
df = pd.concat([df, skill1, skill2], axis = 1)
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell', 'skill_1_level', 'skill_1_name', 'skill_2_level', 'skill_2_name']

In [114]:
df = df.drop(axis = 1, labels = ['skill_1', 'skill_2'])

In [115]:
df = df[['name', 'class', 'specialty', 'skill_1_name', 'skill_1_level', 'skill_2_name','skill_2_level', 'spell']]
df['spell'] = df['spell'].fillna('')
df['skill_2_name'] = df['skill_2_name'].fillna('')
df['skill_2_level'] = df['skill_2_level'].fillna('')

## Creating new columns.

In [116]:
df['count'] = 1
df['spell_tf'] = df['spell'].apply(lambda x: 0 if x == '' else 1)

## Data from other sources

### Data from Towns doc

In [117]:
towns = pd.read_html('http://heroes.thelazy.net/index.php/List_of_all_town_names')
towns = list(towns[1]['Town'].unique())

In [118]:
towns = ['Castle', 'Rampart', 'Tower', 'Inferno', 'Necropolis','Dungeon', 'Stronghold' ,'Fortress','Conflux', 'Cove']

In [119]:
classes = list(df['class'].unique())
classes.remove('Captain')
classes.remove('Navigator')
classes.insert(18, 'Captain')
classes.insert(19, 'Navigator')
classes[8] = 'Death Knight'

In [120]:
towns2 = []
temp_list = []
for i in towns:
  temp_list = [i, i]
  towns2.append(temp_list)
towns2x = [element for sublist in towns2 for element in sublist]
len(classes), len(towns2x)

(20, 20)

In [121]:
class_towns = pd.DataFrame({'class': classes, 'town': towns2x})
df = df.merge(class_towns, how='left', on='class')

In [122]:
df.columns = ['name', 'class', 'specialty', 'skill_1', 'level_1',
       'skill_2', 'level_2', 'spell', 'count', 'spell_tf', 'town']

In [123]:
df['class_2'] = df['skill_1'].apply(lambda x: 'magic' if x == 'Wisdom' else 'might')
df.iloc[81, 11] = 'might'  #Rashka
df.iloc[101, 11] = 'magic' #Aislinn

In [124]:
df = df[['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1',
       'level_2', 'skill_2', 'spell', 'count', 'spell_tf', 'town']]

In [125]:
df.iloc[91:101 , 11] = 'Necropolis'

### Data from creatures_csv

In [126]:
creatures = pd.read_csv('/content/df_h3.csv')
creatures = creatures[['Name', 'Town','Upgraded']]

In [127]:
creatures['specialty'] = creatures['Name'].apply(lambda x: x + 's')
creatures = creatures[creatures['Upgraded'] == 0].reset_index(drop = True)

In [128]:
creatures['specialty'] = creatures['specialty'].apply(lambda x: 'convert' if x.endswith('mans') else x)

## Manually correcting errors

In [129]:
# creatures[creatures['specialty'] == 'convert']
creatures.iloc[0, 3] = 'Pikemen'
creatures.iloc[3, 3] = 'Swordsmen'
creatures.iloc[50, 3] = 'Lizardmen'

creatures.iloc[64, 3] = 'Crew Mate'
creatures.iloc[65, 3] = 'Sea Dogs'
creatures.iloc[67, 3] = 'Sea Witches'
creatures.iloc[68, 3] = 'Nix'

creatures.iloc[8, 3] = 'Dwarves'
creatures.iloc[9, 3] = 'Elves'
creatures.iloc[10, 3] = 'Pegasi'
creatures.iloc[26, 3] = 'Efreet'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[36, 3] = 'Harpies'
creatures.iloc[47, 3] = 'Cyclops'
creatures.iloc[51, 3] = 'Cyclops'

creatures.iloc[11, 3] = 'Dendroids'
creatures.iloc[16, 3] = 'Golems'
creatures.iloc[17, 3] = 'Magi'

# creatures.head(5)

## Merging data (df1)

## *Column on creature specialty*

In [130]:
df2 = df.merge(creatures, how = 'left', on = 'specialty', suffixes = None)
df2 = df2.drop(axis = 0, index = 135).reset_index(drop = True)

## Manually correcting errors

In [131]:
df2.iloc[73,12] = 'Enchanter'
df2.iloc[0,12] = 'Pirate'
df2.iloc[3,12] = 'Crew Mate'
df2.iloc[9,12] = 'Sea Dog'
df2.iloc[96,12] = 'Walking Dead'
df2.iloc[150, 12] = 'Serpent Fly'
df2 = df2.drop(axis = 0, index = 135).reset_index(drop = True)

In [132]:
df2['Name'] = df2['Name'].fillna('')
df2 = df2.drop(axis=1, labels = ['Town', 'Upgraded', 'count', 'spell_tf'])

In [133]:
df2.columns = ['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1', 'level_2', 'skill_2', 'skill_spell', 'town', 'skill_creature']

In [134]:
df2.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,skill_spell,town,skill_creature
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,


## Recreating (df)

In [135]:
df = df2.copy()
df.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,skill_spell,town,skill_creature
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,


## *Column on resource specialty*

In [136]:
df['skill_resource'] = ''
df.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,skill_spell,town,skill_creature,skill_resource
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate,
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph,
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate,
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,,


In [137]:
pattern_resource = '|'.join(['Crystal', 'Mercury', 'Gold', 'Sulfur', 'Gems'])
index_resource = df[df['specialty'].str.contains(pattern_resource)].index.values

In [138]:
df.iloc[index_resource, [11]] = df.iloc[index_resource, [3]].values.flatten().reshape(15,1)

In [139]:
print(df['specialty'].unique())

['Pirates' 'Nymphs' 'Offense' 'Crew Mates' 'Navigation' 'Stormbirds'
 'Cannon' 'Gold' 'Scouting' 'Sea Dogs' 'Nix' 'Crystal' 'Cure'
 'Sea Witches' 'Air Shield' 'Clone' 'Fireball' 'Sorcery' 'Forgetfulness'
 'Ballista' 'Griffins' 'Archery' 'Swordsmen' 'Cavaliers' 'Archers'
 'Estates' 'Speed' 'Bless' 'Frost Ring' 'Weakness' 'Monks' 'Prayer'
 'First Aid' 'Eagle Eye' 'Unicorns' 'Elves' 'Logistics' 'Armorer'
 'Dendroids' 'Resistance' 'Dwarves' 'Sharpshooters' 'Interference'
 'Pegasi' 'Ice Bolt' 'Slayer' 'Intelligence' 'Fortune' 'Nagas' 'Genies'
 'Golems' 'Gargoyles' 'Mercury' 'Hypnotize' 'Haste' 'Mysticism'
 'Chain Lightning' 'Magi' 'Enchanters' 'Gogs' 'Hell Hounds' 'Imps'
 'Demons' 'Pit Fiends' 'Efreet' 'Devils' 'Bloodlust' 'Sulfur'
 'Inferno (spell)' 'Wights' 'Skeletons' 'Necromancy' 'Liches'
 'Walking Dead' 'Black Knights' 'Vampires' 'Meteor Shower' 'Death Ripple'
 'Animate Dead' 'Stone Skin' 'Beholders' 'Minotaurs' 'Harpies'
 'Troglodytes' 'Manticores' 'Dragons' 'Resurrection' 'Goblins' '

## *Column on secondary skill specialty*

Some heroes specialize in a secondary skill, like Necromancy, Armorer, Navigation, Offense etc.

## *Column on spell skill specialty*

Some heroes specialize in casting a skill, like Ressurection, Inferno etc.

In [143]:
spell_df = pd.read_html('https://heroes.thelazy.net/index.php/List_of_spells')[0]

In [152]:
spell_df.columns = ['skill_spell', 'spell_level', 'spell_class', 'effect']

In [153]:
df = df[['name', 'class', 'class_2', 'town','specialty', 'level_1', 'skill_1','level_2', 'skill_2', 'skill_creature','skill_resource', 'skill_spell']]
df = df.merge(spell_df, how='left', on='skill_spell')
df = df.fillna('').drop('effect', axis = 1)

In [154]:
df['spell_level'] = df['spell_level'].apply(lambda x: int(x) if isinstance(x, float) else 0)

In [198]:
df[df['spell_class'].str.contains('All Schools')]

,name,class,class_2,town,specialty,level_1,skill_1,level_2,skill_2,skill_creature,skill_resource,skill_spell,spell_level,spell_class
54,Malcom,Druid,magic,Rampart,Eagle Eye,Basic,Wisdom,Basic,Eagle Eye,,,Magic Arrow,1,All Schools
58,Iona,Alchemist,might,Tower,Genies,Basic,Scholar,Basic,Intelligence,Genie,,Magic Arrow,1,All Schools
62,Rissa,Alchemist,might,Tower,Mercury,Basic,Mysticism,Basic,Offense,,Mercury,Magic Arrow,1,All Schools
63,Thane,Alchemist,might,Tower,Genies,Advanced,Scholar,,,Genie,,Magic Arrow,1,All Schools
64,Torosar,Alchemist,might,Tower,Ballista,Basic,Mysticism,Basic,Tactics,,,Magic Arrow,1,All Schools
91,Charna,Death Knight,might,Necropolis,Wights,Basic,Necromancy,Basic,Tactics,Wight,,Magic Arrow,1,All Schools
92,Clavius,Death Knight,might,Necropolis,Gold,Basic,Necromancy,Basic,Offense,,Gold,Magic Arrow,1,All Schools
94,Isra,Death Knight,might,Necropolis,Necromancy,Advanced,Necromancy,,,,,Magic Arrow,1,All Schools
97,Tamika,Death Knight,might,Necropolis,Black Knights,Basic,Necromancy,Basic,Offense,Black Knight,,Magic Arrow,1,All Schools
117,Mutare,Overlord,might,Dungeon,Dragons,Basic,Estates,Basic,Tactics,,,Magic Arrow,1,All Schools


In [204]:
mask = (df['town']=='Dungeon')
type(mask)
df[mask]

,name,class,class_2,town,specialty,level_1,skill_1,level_2,skill_2,skill_creature,skill_resource,skill_spell,spell_level,spell_class
109,Ajit,Overlord,might,Dungeon,Beholders,Basic,Leadership,Basic,Interference,Beholder,,,0,
110,Arlach,Overlord,might,Dungeon,Ballista,Basic,Offense,Basic,Artillery,,,,0,
111,Dace,Overlord,might,Dungeon,Minotaurs,Basic,Tactics,Basic,Offense,Minotaur,,,0,
112,Damacon,Overlord,might,Dungeon,Gold,Advanced,Offense,,,,Gold,,0,
113,Gunnar,Overlord,might,Dungeon,Logistics,Basic,Tactics,Basic,Logistics,,,,0,
114,Lorelei,Overlord,might,Dungeon,Harpies,Basic,Leadership,Basic,Scouting,Harpy,,,0,
115,Shakti,Overlord,might,Dungeon,Troglodytes,Basic,Offense,Basic,Tactics,Troglodyte,,,0,
116,Synca,Overlord,might,Dungeon,Manticores,Basic,Leadership,Basic,Scholar,Manticore,,,0,
117,Mutare,Overlord,might,Dungeon,Dragons,Basic,Estates,Basic,Tactics,,,Magic Arrow,1,All Schools
118,Mutare Drake,Overlord,might,Dungeon,Dragons,Basic,Estates,Basic,Tactics,,,Magic Arrow,1,All Schools


In [203]:
pd.pivot_table(df, index = 'town',  columns = 'spell_class', values = 'spell_level', aggfunc = 'count', fill_value = '', margins = False)

spell_class,,Air Magic,All Schools,Earth Magic,Fire Magic,Water Magic
town,,,,,,
Castle,12.0,,,1.0,1,6.0
Conflux,8.0,2.0,1.0,2.0,2,1.0
Cove,11.0,1.0,,1.0,1,5.0
Dungeon,8.0,1.0,2.0,6.0,1,
Fortress,8.0,,1.0,5.0,2,2.0
Inferno,9.0,2.0,,2.0,3,1.0
Necropolis,,3.0,4.0,10.0,1,
Rampart,11.0,2.0,1.0,,2,2.0
Stronghold,9.0,3.0,1.0,2.0,2,


In [205]:
dfu = pd.pivot_table(df, index = 'town',  columns = 'spell_class', values = 'spell_level', aggfunc = 'count', fill_value = '', margins = False)

In [206]:
dfu = dfu.reset_index()
dfu.shape

(10, 6)

In [236]:
dfu.fillna(0, inplace = True)
dfu.head()
dfu.replace({'': 0})

,town,na,Air Magic,All Schools,Earth Magic,Fire Magic,Water Magic
0,Castle,12.0,0.0,0.0,1.0,1,6.0
1,Conflux,8.0,2.0,1.0,2.0,2,1.0
2,Cove,11.0,1.0,0.0,1.0,1,5.0
3,Dungeon,8.0,1.0,2.0,6.0,1,0.0
4,Fortress,8.0,0.0,1.0,5.0,2,2.0
5,Inferno,9.0,2.0,0.0,2.0,3,1.0
6,Necropolis,0.0,3.0,4.0,10.0,1,0.0
7,Rampart,11.0,2.0,1.0,0.0,2,2.0
8,Stronghold,9.0,3.0,1.0,2.0,2,0.0
9,Tower,0.0,7.0,4.0,4.0,1,1.0


In [237]:
dfu.plot(kind = 'barh', y = 'town', x='Air Magic')

TypeError: ignored

## *Column on war machines specialty*

In [156]:
df[(df['specialty'] == 'Cannon') | (df['specialty'] == 'Ballista')]

,name,class,class_2,town,specialty,level_1,skill_1,level_2,skill_2,skill_creature,skill_resource,skill_spell,spell_level,spell_class
6,Jeremy,Captain,might,Cove,Cannon,Basic,Offense,Basic,Artillery,,,,0,
19,Christian,Knight,might,Castle,Ballista,Basic,Leadership,Basic,Artillery,,,,0,
64,Torosar,Alchemist,might,Tower,Ballista,Basic,Mysticism,Basic,Tactics,,,Magic Arrow,1,All Schools
80,Pyre,Demoniac,might,Inferno,Ballista,Basic,Logistics,Basic,Artillery,,,,0,
100,Ranloo,Death Knight,might,Necropolis,Ballista,Basic,Necromancy,Basic,Artillery,,,Haste,1,Air Magic
110,Arlach,Overlord,might,Dungeon,Ballista,Basic,Offense,Basic,Artillery,,,,0,
129,Gurnisson,Barbarian,might,Stronghold,Ballista,Basic,Offense,Basic,Artillery,,,,0,
148,Gerwulf,Beastmaster,might,Fortress,Ballista,Basic,Armorer,Basic,Artillery,,,,0,


In [157]:
df[df['town']=='Inferno']

,name,class,class_2,town,specialty,level_1,skill_1,level_2,skill_2,skill_creature,skill_resource,skill_spell,spell_level,spell_class
74,Calh,Demoniac,might,Inferno,Gogs,Basic,Archery,Basic,Scouting,Gog,,,0,
75,Fiona,Demoniac,might,Inferno,Hell Hounds,Advanced,Scouting,,,Hell Hound,,,0,
76,Ignatius,Demoniac,might,Inferno,Imps,Basic,Tactics,Basic,Interference,Imp,,,0,
77,Marius,Demoniac,might,Inferno,Demons,Advanced,Armorer,,,Demon,,,0,
78,Nymus,Demoniac,might,Inferno,Pit Fiends,Advanced,Offense,,,Pit Fiend,,,0,
79,Octavia,Demoniac,might,Inferno,Gold,Basic,Scholar,Basic,Offense,,Gold,,0,
80,Pyre,Demoniac,might,Inferno,Ballista,Basic,Logistics,Basic,Artillery,,,,0,
81,Rashka,Demoniac,might,Inferno,Efreet,Basic,Scholar,Basic,Wisdom,Efreeti,,,0,
82,Xeron,Demoniac,might,Inferno,Devils,Basic,Leadership,Basic,Tactics,Devil,,,0,
83,Ash,Heretic,magic,Inferno,Bloodlust,Basic,Wisdom,Basic,Eagle Eye,,,Bloodlust,1,Fire Magic
